# Tugas Pembelajaran Mendalam (MA4072)
## Reinforcement Learning: Q-Learning
Kamis, 29 April 2021
> Membuat agen yang bergerak maju, belok kanan, dan belok kiri dalam lingkungan 2D.

> Dibuat oleh Ilma Aliya Fiddien (10117019).

In [2]:
import numpy as np
import pandas as pd
import random
from time import time
import sys
import copy

In [3]:
# Memodelkan layout sederhana lantai 5 gedung CAS
rooms = ['I', 'W', 'K', 'TU', 'D', 'R1', 'U', 'L', 'N', 'R2', 'H', 'E']

env = np.array([
    ['H',  0, 'E'], # H  E
    ['R2', -1,  0], # R2
    ['U', 'L','N'], # U L N
    [  0, -1,'R1'], # R1
    ['K','TU','D'], # K TU D
    [   0, -1,  0],
    [ 'I',  0,'W'], # I W
])

reward = np.array([
    [  1,0.5,  1,], # H  E
    [  1, -1,0.5,], # R2
    [  1,  1,  1,], # U L N
    [0.5, -1,  1,], # R1
    [  1,  1,  1,], # K TU D
    [0.5, -1,0.5,],
    [  1,0.5,  1,], # I W
])

def color_env(val):
    if val < 0 : color = 'black'
    elif val == 1: color = 'green'
    else: color = 'red'
    return 'color: %s' % color

def color_Q(val):
    if val != 0: color = 'green'
    else: color = 'black'
    return 'color: %s' % color

def color_env_(val):
    if val in rooms : color = 'red'
    elif val == '0' : color = 'green'
    else: color = 'black'
    return 'color: %s' % color

# display(pd.DataFrame(reward).style.applymap(color_env))
display(pd.DataFrame(env).style.applymap(color_env_))


# mendefinisikan pemetaan dari state ke posisi dan sebaliknya
state_pos = {}; pos_state = {}
k = 0
for i in range(reward.shape[0]):
    for j in range(reward.shape[1]):
        pos_state[k] = (i, j)
        state_pos[(i,j)] = k
        k += 1
        
# mendefinisikan pemetaan dari nama ruangan ke posisi dan sebaliknya
pos_room = {} ; room_pos = {}
actionable_states = []
for i in range(reward.shape[0]):
    for j in range(reward.shape[1]):
        if reward[i,j] >= 0:
            actionable_states.append(state_pos[(i,j)])
            if env[i,j] in rooms:
                pos_room[env[i,j]] = (i,j)
                room_pos[(i,j)] = env[i,j]

def to_state(x):
    if type(x) == tuple:
        return state_pos[x]
    else:
        return state_pos[pos_room[x]]

def to_pos(x):
    if type(x) == int:
        return pos_state[x]
    else:
        return pos_room[x]
    
def to_room(x):
    if type(x) == tuple:
        return room_pos[x]
    else:
        return room_pos[pos_state[x]]
    
def reward_reset():
    return reward    

print("State yang merupakan ruang kantor:\n", pos_room)
print("State yang bisa dipilih:\n", actionable_states)
print("Banyaknya state yang bisa dipilih: ",len(actionable_states))
print("Banyak state (total): ",len(state_pos))

,0,1,2
0,H,0,E
1,R2,-1,0
2,U,L,N
3,0,-1,R1
4,K,TU,D
5,0,-1,0
6,I,0,W


State yang merupakan ruang kantor:
 {'H': (0, 0), 'E': (0, 2), 'R2': (1, 0), 'U': (2, 0), 'L': (2, 1), 'N': (2, 2), 'R1': (3, 2), 'K': (4, 0), 'TU': (4, 1), 'D': (4, 2), 'I': (6, 0), 'W': (6, 2)}
State yang bisa dipilih:
 [0, 1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 18, 19, 20]
Banyaknya state yang bisa dipilih:  18
Banyak state (total):  21


Di sini, arah "maju", "belok kanan", dan "belok kiri" akan dipandu oleh empat arah mata angin. Gerakan "mundur" atau "putar balik" akan dicegah pada pemilihan `action` di algoritma selanjutnya.

In [4]:
# memetakan kolom tabel Q ke aksi (bilangan bulat)
# pemilihan angka-angka aksi untuk mengetahui arah berlawanan
A = { 0:-1, # utara/atas
      1: 1, # selatan/bawah
      2:-2, # barat/kiri
      3: 2 }# timur/kanan

# untuk keperluan printing
direction = { 
      0: 'utara', 
      1: 'selatan',
      2: 'barat', 
      3: 'timur' }

In [5]:
class Q_learning():
    def __init__(self, target, print_status, discount = 0.9, lr = 0.1, epochs = 20000):
        self.reward = reward_reset()
        x,y = to_pos(target)
        self.reward[x, y] = 100
        
        self.target = to_state(target)
        self.table = self.training(discount, lr, epochs, print_status)
    
    def R(self, pos):
        x, y = pos
        if x > self.reward.shape[0]-1 or y > self.reward.shape[1]-1 or x < 0 or y < 0:
            return -1
        else:
            return self.reward[x,y]

    def q_table_initialize(self):
        ''' inisialisasi tabel Q'''
        Q = np.zeros((len(pos_state.keys()), len(A)))
        return Q

    def get_all_possible_next_action(self):
        ''' mendaftar semua arah action selanjutnya yang mungkin
            pada state sekarang '''
        (x, y) = pos_state[self.cur_state]
        around = {0: self.R([x-1, y]), 
                  1: self.R([x+1, y]), 
                  2: self.R([x, y-1]), 
                  3: self.R([x, y+1])}
        return [k for k,v in around.items() if v >= 0]

    def get_next_state(self, action):
        ''' mendapatkan state selanjutnya, 
            diberikan action yang dipilih pada state sekarang '''
        (x, y) =  to_pos(self.cur_state)
        if action == 0:
            return to_state((x-1, y))
        elif action == 1:
            return to_state((x+1, y))
        elif action == 2:
            return to_state((x, y-1))
        elif action == 3:
            return to_state((x, y+1))

    def is_finished(self):
        ''' mengecek apakah sudah mencapai target '''
        return self.cur_state == self.target

    def training(self, d, lr, epochs, print_status): 
        tic = time()
        Q = self.q_table_initialize()
        for _ in range(epochs):
            # memilih tempat memulai latihan
            self.cur_state = random.choice(actionable_states)

            # ketika belum sampai target
            count = 0
            while(not self.is_finished()):
                # mendapatkan setiap kemungkinan arah aksi selanjutnya
                possible_actions = self.get_all_possible_next_action()

                # pilih aksi mana saja secara random
                backward = True
                while(backward): # tapi tidak boleh berbalik arah
                    action = random.choice(possible_actions)
                    if count == 0 or A[action] != -A[prev_action]:
                        backward = False

                # menemukan state selanjutnya akibar dari aksi yang dipilih
                next_state = self.get_next_state(action)
                
                s = self.cur_state
                # memperbarui tabel Q
                TD = self.R(to_pos(s)) + (d * max(Q[s])) - Q[s][action]
                Q[s][action] += lr * TD

                # lanjut ke state berikutnya
                self.cur_state = next_state
                prev_action = action
                count += 1

            # print status
            if print_status:
                toc = time()
                sys.stdout.write("\rEpoch "+str(_+1)+" selesai | "+str(round(toc-tic))+"s---")
        if print_status:
            print("\nSelesai berlatih!")

        return Q


    def find_path(self, start):
        ''' melakuakan inferensi dengan tabel Q yang telah diisi'''
        path = [start]
        init_state = to_state(start)
        self.cur_state  = init_state
        
        print(f"Dari {start} di {to_pos(init_state)}")
        finish = self.target
        j = 0
        
        # jika belum mencapai target
        while (self.cur_state != finish):  
            
            # pilih aksi
            s = self.cur_state
            action = np.argmax(self.table[s,:])
            if j == 0: prev_action = action
            if A[action] == -A[prev_action]:
                action = self.table[s,:].argsort()[-3:][::-1][1]
            
            # rekam aksi untuk memori
            prev_action = action
            
            # dapatkan state dari aksi
            next_state = self.get_next_state(action) 
            self.cur_state   = next_state
            
            # tambahkan state ke lintasan
            if pos_state[next_state] in room_pos:
                path.append(to_room(next_state))
            else:
                path.append(to_pos(next_state))
            j += 1
            print(f"Langkah ke-{j}, berjalan ke arah {direction[action]}, tiba di {path[-1]}")
            if j > 25:  # belum pernah sampai target
                print('Belum sampai target, perbesar epochs!')
                break
        self.path = path

# Contoh

In [9]:
discount = 0.9
learning_rate = 0.1
epochs = 5000
args = (discount, learning_rate, epochs)

### Mencari lintasan dari ruangan Bu Nuning ke ruangan Pak Hendra

In [10]:
awal = 'N'
akhir = 'H'
Q1 = Q_learning(akhir, True, *args)
Q1.find_path(awal)

Epoch 5000 selesai | 1s---
Selesai berlatih!
Dari N di (2, 2)
Langkah ke-1, berjalan ke arah utara, tiba di (1, 2)
Langkah ke-2, berjalan ke arah utara, tiba di E
Langkah ke-3, berjalan ke arah barat, tiba di (0, 1)
Langkah ke-4, berjalan ke arah barat, tiba di H


### Mencari lintasan dari ruangan Bu Nuning ke ruangan Pak Hendra, tetapi mampir dulu ke ruangan Pak Wono

In [11]:
awal = 'N'
tengah = 'W'
akhir = 'H'
Q2 = Q_learning(tengah, False, *args)
Q2.find_path(awal)
Q3 = Q_learning(akhir, False, *args)
Q3.find_path(tengah)

Dari N di (2, 2)
Langkah ke-1, berjalan ke arah selatan, tiba di R1
Langkah ke-2, berjalan ke arah selatan, tiba di D
Langkah ke-3, berjalan ke arah selatan, tiba di (5, 2)
Langkah ke-4, berjalan ke arah selatan, tiba di W
Dari W di (6, 2)
Langkah ke-1, berjalan ke arah barat, tiba di (6, 1)
Langkah ke-2, berjalan ke arah barat, tiba di I
Langkah ke-3, berjalan ke arah utara, tiba di (5, 0)
Langkah ke-4, berjalan ke arah utara, tiba di K
Langkah ke-5, berjalan ke arah utara, tiba di (3, 0)
Langkah ke-6, berjalan ke arah utara, tiba di U
Langkah ke-7, berjalan ke arah utara, tiba di R2
Langkah ke-8, berjalan ke arah utara, tiba di H


In [12]:
display(pd.DataFrame(Q1.table, columns=direction.values()).style.applymap(color_Q))
display(pd.DataFrame(Q2.table, columns=direction.values()).style.applymap(color_Q))
display(pd.DataFrame(Q3.table, columns=direction.values()).style.applymap(color_Q))

,utara,selatan,barat,timur
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,5.000000,4.999997
2,0.000000,10.000000,10.000000,0.000000
3,10.000000,9.999956,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000
5,5.000000,5.000000,0.000000,0.000000
6,10.000000,10.000000,0.000000,10.000000
7,0.000000,0.000000,10.000000,10.000000
8,10.000000,10.000000,10.000000,0.000000
9,5.000000,5.000000,0.000000,0.000000


,utara,selatan,barat,timur
0,0.000000,1000.000000,0.000000,1000.000000
1,0.000000,0.000000,5.000000,5.000000
2,0.000000,10.000000,10.000000,0.000000
3,10.000000,10.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000
5,5.000000,5.000000,0.000000,0.000000
6,10.000000,10.000000,0.000000,10.000000
7,0.000000,0.000000,10.000000,10.000000
8,10.000000,10.000000,10.000000,0.000000
9,5.000000,5.000000,0.000000,0.000000


,utara,selatan,barat,timur
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,5.000000,4.999998
2,0.000000,10.000000,10.000000,0.000000
3,10.000000,9.999958,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000
5,5.000000,5.000000,0.000000,0.000000
6,10.000000,10.000000,0.000000,10.000000
7,0.000000,0.000000,10.000000,10.000000
8,10.000000,10.000000,10.000000,0.000000
9,5.000000,5.000000,0.000000,0.000000
